In [1]:
from pathlib import Path 
import random

import polars as pl

In [2]:
basepath = Path("/users/cn/itrujnara/genomes-project/infer_chromosome/")

In [3]:
df_seq = pl.read_parquet(basepath / "one_chunk/random_chunk.pq")

In [4]:
df_seq

id,description,sequence
str,str,str
"""random_chunk""","""<unknown description>""","""ATGTAAGAGAAGAAAACTTTGGCCAAGAAT…"


In [5]:
seq = df_seq.item(0,2)

In [6]:
len(seq)

131072

In [18]:
size = 131072 // 4 + 8192
size

40960

In [19]:
stride = size - 8192
stride

32768

In [20]:
chunks = []

for i,start in enumerate(range(0,len(seq),stride)):
    chunk_seq = seq[start:start+size]
    chunks.append({
        "id": f"random_chunk_{i}",
        "start": start,
        "end": min(start+size, len(seq)),
        "sequence": chunk_seq
    })

df_chunks = pl.DataFrame(chunks)

In [21]:
df_chunks

id,start,end,sequence
str,i64,i64,str
"""random_chunk_0""",0,40960,"""ATGTAAGAGAAGAAAACTTTGGCCAAGAAT…"
"""random_chunk_1""",32768,73728,"""AAGAAGGAACACAGGACCGTAAACAGCTTT…"
"""random_chunk_2""",65536,106496,"""AAGTCTTAACTCATTTCAGTATTAACCCAA…"
"""random_chunk_3""",98304,131072,"""ACATTTATACCTATGTAACAAACCTACACG…"


In [34]:
random.seed(42)
random_chunk = df_chunks[random.randint(0,df_chunks.shape[0]-1)]
random_chunk

id,start,end,sequence
str,i64,i64,str
"""NC_000021.9""",41615360,42663936,"""AGTGAGTTCGAGGCTTCAGTGAGCTGTGAC…"


In [23]:
prefix = "random_chunk"

In [38]:
random_chunk.write_parquet(basepath / f"one_chunk/{prefix}_random_long.pq")

In [24]:
df_chunks.write_parquet(basepath / "one_chunk" / f"{prefix}_chunks_{size}.pq")